# San Diego-Tijuana OSM Data Wrangling
#####Daniel Cohen  
#####Udacity Data Analyst Nanodegree Project 3
##Overview
The purpose of this project was to take an area of data from the OpenStreetMaps project and wrangle that data into a clean form which could be imported into a database and queried. I choose San Diego, because that is where I live. After downloading the data, a ran several python scripts to audit and clean the data. The dataset was then imported into MongoDB and queried for some knowledge as to its structure and quality. The following document explains problems that arose when auditing the data, how these were solved to clean the data, and what some initial data querying showed. All the python scripts and text output files mentioned are located in the github repo.  
Map Area: San Diego, CA, USA and Tijuana, Mexico  
http://osm-extracted-metros.s3.amazonaws.com/samara-tolyatti.osm.bz2
https://www.openstreetmap.org/api/0.6/relation/253832

##Problems Encountered With Map Data

My general approach for wrangling the data was to run python scripts in the terminal and output the results to text files which could be easily skimmed to view problematic areas. Once I had run a thorough audit and found all the areas to clean, I parsed the xml data and converted it to json format, cleaning the necessary items as they came up (data.py script). I then loaded the clean json document into MongoDB. I ran queries against this database to check for quality, and if I found any issues, then I went back to the audit.py and data.py files to create a new clean json file and continued to iterate until a quality, clean database was achieved.


###Initial auditing
Upon first auditing the data, I started with the same tasks as in problem set 6 of the Data Wrangling course. These included the following:
- getting a list of unique tags (i.e. way, node, etc.) (mapparser.py)
- examining how many tag attributes had problematic characters in their keys (tags.py)
- looking at unique users who had inputed data (users.py)

####Types of tags
I ran the following line in the terminal bash shell to get a list of tag types
> $ python mapparser.py san-diego-tijuana_mexico.osm > uniqueTags.txt

The result showed no unexpected tags:
{'bounds': 1,
 'member': 7719,
 'nd': 1203075,
 'node': 1510747,
 'osm': 1,
 'relation': 791,
 'tag': 5925330,
 'way': 117899}

####Search for problematic keys
Next I ran a check for problematic key labels. A key that contains a problematic character such as a '(' might cause problems when inputting the data into MongoDB
> $ python tags.py san-diego-tijuana_mexico.osm > keyTypes.txt

{'lower': 850599, 'lower_colon': 5035595, 'other': 39128, 'problemchars': 8}

There were only 8 tags that had problematic characters in their key values, a very low percent, considering there are over 5 million tags. Most tags keys have a colon, which could be something like 'addr:street'.

Printing out the problematic keys showed that there was typically a blank space or a '.' in the key, but since there were so few, I chose to omit these when processing the data into the database.

####Unique users
Lastly, I ran a check to see how many unique users had inputted data into the dataset.
> $ python users.py san-diego-tijuana_mexico.osm > uniqueUsers.txt

This script showed there were 1116 unique users who had contributed to the dataset.

###Auditing more fields

####Street types
The initial scripts provided a good overview of the data but more digging was necessary to see where problems would crop up. I looked into the street names, anticipating that this was a field that could use some cleaning. Sure enough, there were many street types that were unexpected. Some of these were simple abbreviations or mispellings that could be mapped to an appropriate name. Others were not expected but correct names, such as 'highway 101'. Still others were in Spanish, which is common in San Diego, due to the proximity to Mexico and influence of Spain in the city's history (also some locations are actually in Mexico, as indicated from the osm file's title). Names starting with "Avenida" or "Camino" are correct - the street type typically comes _before_ the street name in Spanish - so these needed a slightly different approach to cleaning. I used a dictionary of expected Spanish street types and a mapping dictionary, searched the beginning of the street attribute with a regex and found the abbreviations/mispellings that could be fixed easily here (e.g. "Camto" or "Aveinda"). I also included the cardinal directions to replace with the full word (i.e. 'West' for 'W') for consistency. However, there is both an E street and an East street in San Diego, so I had to be careful there. This fixed the large majority of street names.

####Zip codes
Postal codes were another area of data that needed some cleaning, after initial auditing found some containing 4 number extensions ('92071-4417'), some prefixed with 'CA' and others that were inapproriate (either containing wrong number of digits or only containing the city name, for example). I wrote clean_zip function to put everything in 5 digit format and ignore the rest, also checking that all codes were in fact for the San Diego/Tijuana region (a couple were outside of this area - those nodes should be excluded based on lat/lon data).

####other geographic data
Many nodes had tags with "is_in" or "gnis" attributes, which are structured similarly to the address data (and may be somewhat redundant), having subfields denoted with a colon (e.g. is_in:country). Although I am unsure of the exact use, I did not want to ignore this data. I decided to treat it similarly to the address attribute and create a subdocument within each node containing any is_in tags, and another for gnis tags. But first I audited these tags to see what those subfields were and if there were any cleaning tasks required.

For the is_in tags, the data was fairly clean to begin with. The biggest problem I saw was the presence of a general is_in tag (that is, no subfield following preceded by a colon), which contained various lists of anything from just the city name to 'San Diego, California, Calif., CA, USA'. There were also some redundant fields (state_code and iso_3166_2, for example) and inconsistent data values (USA vs United States of America) that could use cleaning. I mapped the is_in data to it's appropriate field (e.g. putting San Diego under the city subfield), and fixed any inconsistencies in field or value. This is done in the data.py script and the clean_is_in function.

The gnis data was also fairly clean. A few issues were: 
- Some 'created' data had multiple values
- A few ele data points are in the wrong units (ft instead of meters, as per OSM docs)
- There are both 'feature_id' and 'id' subfields, which should be the same thing, and some of these have multiple values

I decided to only take the first value from the created field, to get rid of units in the ele tags (this will cause a few errors but I am assuming most elevations are listed in the appropriate units of meters), and to combine feature_id and id into one field and make it array valued, since maybe a node could belong to multiple geographic feature ids. These fixes are contained in the data.py script and the clean_gnis function.

###Auditing Summary
After all auditing and cleaning steps, street names and zip codes should be much more consistent. Address, GNIS and is_in data are reformatted into subdocuments, and the fields within these have undergone some modifications. Finally, only nodes and ways were converted into JSON, as in the course problem set. This mean all relation tags were dropped before importing into the database.


##Data Overview

Once the data was audited and cleaning procedures established, I ran the following lines in the bash shell to convert to JSON and import into MongoDB.
> `$` python data.py san-diego-tijuana_mexico.osm  
`$` mongoimport -d OSM -c SDT --file san-diego-tijuana_mexico.osm.json

####File Size
san-diego-tijuana_mexico.osm = 538.2 MB  
san-diego-tijuana_mexico.osm.json = 515.1 MB

####Number of documents
\> db.SDT.find().count()  
1628646

####Number of Unique Users
\> db.SDT.distinct("created.uid").length  
1109  
This is just off from the number of unique users found by auditing the initial xml file (1116), but the missing 5 could have been users that only created/edited relation tags, since those were not imported into the database.

####Number of nodes and ways
\> db.SDT.find({'type' : 'node'}).count()  
1510747  
\> db.SDT.find({'type' : 'way'}).count()  
117899  
So there are a total of 1628646 nodes and ways which agrees with the total number of documents. Put another way, no document has a type other than node or way, which is another good consistency check.

####Most frequent user
\> db.SDT.aggregate([{"`$`group" : {"_id" : "`$`created.user", "count" : {"`$`sum" : 1}}}, {"`$`sort" : {"count" : -1}}, {"`$`limit" : 1}])  
{ "_id" : "Adam Geitgey", "count" : 681737 }  

####Number of users with one post
\> db.SDT.aggregate([{"`$`group":{"_id":"`$`created.user", "count":{"`$`sum":1}}}, {"`$`group":{"_id":"`$`count", "num_users":{"`$`sum":1}}}, {"`$`sort":{"_id":1}}, {"`$`limit":1}])  
{ "_id" : 1, "num_users" : 200 }  
200 users only submitted one edit

####Most frequent streets
\> db.SDT.aggregate([{"`$`match":{"address.street": {"`$`exists":1}}}, {"`$`group":{"_id" : "`$`address.street", "count":{"`$`sum":1}}}, {"`$`sort":{"count":-1}}, {"`$`limit":5}])  
{ "_id" : "Broadway", "count" : 963 }  
{ "_id" : "El Cajon Boulevard", "count" : 744 }  
{ "_id" : "University Avenue", "count" : 712 }  
{ "_id" : "Central Avenue", "count" : 601 }  
{ "_id" : "Madison Avenue", "count" : 596 }  
These are the most frequently occuring streets. This makes sense since these streets are fairly long and run through major commercial centers of San Diego, so there are a lot of interesting nodes to tag along them.

####Counts of common ammenities
\> db.SDT.aggregate([{"`$`match":{"amenity": {"`$`exists":1}}}, {"`$`group":{"_id" : "`$`amenity", "count":{"`$`sum":1}}}, {"`$`sort":{"count":-1}}, {"`$`limit":10}])  
{ "_id" : "parking", "count" : 2410 }  
{ "_id" : "place_of_worship", "count" : 1076 }  
{ "_id" : "school", "count" : 829 }  
{ "_id" : "fast_food", "count" : 743 }  
{ "_id" : "restaurant", "count" : 669 }  
{ "_id" : "bar", "count" : 294 }  
{ "_id" : "fuel", "count" : 251 }  
{ "_id" : "cafe", "count" : 222 }  
{ "_id" : "bank", "count" : 194 }  
{ "_id" : "toilets", "count" : 144 }  
The list above shows the ten most commonly tagged types of amenities around San Diego. Parking is the most common, and there are almost 300 bars! 

#### Most Popular Cuisine
\> db.SDT.aggregate([{"`$`match":{"amenity": {"`$`exists":1}, "amenity":"restaurant"}}, {"`$`group":{"_id" : "`$`cuisine", "count":{"`$`sum":1}}}, {"`$`sort":{"count":-1}}, {"`$`limit":10}])

{ "_id" : null, "count" : 269 }  
{ "_id" : "mexican", "count" : 65 }  
{ "_id" : "pizza", "count" : 46 }  
{ "_id" : "american", "count" : 34 }  
{ "_id" : "italian", "count" : 29 }  
{ "_id" : "chinese", "count" : 27 }  
{ "_id" : "sushi", "count" : 25 }  
{ "_id" : "burger", "count" : 20 }  
{ "_id" : "thai", "count" : 19 }  
{ "_id" : "japanese", "count" : 17 }  

A lot of restaurants are missing a cuisine type, but then mexican, pizza, and american are some of the most common cuisines.


##Additional Ideas

###Dataset Exploration
This dataset could be really useful for exploring the San Diego area. One potential use would be find the areas with the most number and diversity of amenities, to find a desirable place to live. This is similar to the "walking score" developed by several house/apartment finding websites, but the advantage of having this dataset is that one could customize the criteria - only using the amenities one is most interested in or requiring a specific type of church, for example. Similarly, if this dataset could be a used in the backend of a yelp-type app that takes a specific location and finds things of interest in the area.

This dataset could also find use as a research tool for comparing neighborhoods within San Diego. One could look at how different neighborhoods are growing, where new businesses, schools, etc are being built, or which areas have/lack access to certain amenities, like grocery stores. This could be a tool for businesses, targetting where to build a new restaurant, for example, or the public sector, ensuring all areas have access to fresh foods. The local government could use this data to influence policy about zoning/construction. The most challenging aspect of these applications would be deciphering the key metrics and figuring out how the data relates to teh problem being addressed.

If similar processing and cleaning steps were taken for other cities, one could compare multiple cities in terms of growth, desirability, maybe even come of with models to determine cost of living or job opportunity. 

###Improving the dataset
There is still plenty of room for improving this dataset, both in terms of the raw data, and in restructuring/cleaning to make it more easily queried. The gnis and is_in data, for example, seems to be an area that is largely redundant, and may be able to be consolidated into a single subdocument with the necessary info. This would be cahllenging, though, because it would require a good amount of knowledge as to what these data fields were initially intended for, and one would need to deal with any inconsistencies between the two types (lots of documents have both types of fields). 

The OSM data is definitely incomplete as well. One idea for getting more contributions would be to have people identify images of local places as a way to verify their identity, similar to how currently a lot of websites use a fuzzy word to ensure that a real person and not a bot is, for example, making a purchase. So instead, the user could have to identify a picture (from google maps street view) of a restaurant and have to give its name (or other piece of data), then on the back end, the lat/lon could be queried and updated with the restaurant name, type, or other data. Of course, one would need to aggregate the results of lots of users for accuracy. This would certainly be challenging to set up, and I'm unsure how you would select which pictures to show, but it has potential for getting a lot more users to enter data (only ~1000 users entering data in a San Diego area of >3 million people is not very much).

##Conclusion
This project took a dataset from the OpenStreetMaps project, downloaded, audited, cleaned, and imported the data into MongoDB. Auditing showed some inconsistensies in areas like street type and zip code that were fixed programatically when iterating through the XML file. Some fields (address, created, is_in, and gnis) were reformatted as subdocuments to give the data a clearer hierarchical structure. Once the dataset was wrangled and loaded into the database, queries were run to see a high level overview of the data, inlcuding number of users, nodes, and ways. Finally, some ideas were presented for further uses of this data and possibilities for improving the quality and completeness of the data.
